# MGIMO intensive

## Relational databases practice - 1

### 1. Libraries

In [ ]:
!pip install pyarrow
!pip install pandas

In [ ]:
import os
import sys
import datetime
import pandas as pd

pd.set_option('display.max_columns', None)

### 2. Raw data

In [ ]:
!ls -la

In [ ]:
!ls -la data_section35_112_v20250918_section_file/

In [ ]:
df = pd.read_csv(
    './data_section35_112_v20250918_section_file/data_section35_112_v20250918.csv', 
    sep=';', 
    index_col=False
)
df.info()

In [ ]:
df.head()

In [ ]:
df.describe().T

### 3. Normalize data

#### 3.1. Formulate the task

Попросим ИИ дать нам идеи по решению задачи.

```prompt
Посмотри на структуру датафрейма и дай советы по загрузке этих данных в реляционную базу данных в виде нескольких нормализованных таблиц. 

---
Структура датафрейма:

<class 'pandas.DataFrame'>
RangeIndex: 178475 entries, 0 to 178474
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   indicator_section_code  178475 non-null  int64  
 1   indicator_section       178475 non-null  str    
 2   indicator_code          178475 non-null  str    
 3   indicator_name          178475 non-null  str    
 4   zagr                    178475 non-null  str    
 5   region_id               178475 non-null  int64  
 6   region_name             178475 non-null  str    
 7   mun_level               178475 non-null  str    
 8   mun_district            178475 non-null  str    
 9   municipality            178475 non-null  str    
 10  oktmo                   178475 non-null  int64  
 11  mun_type                178475 non-null  str    
 12  mun_type_oktmo          178475 non-null  str    
 13  oktmo_stable            178475 non-null  str    
 14  oktmo_history           178475 non-null  str    
 15  oktmo_year_from         178475 non-null  str    
 16  oktmo_year_to           178475 non-null  str    
 17  year                    178475 non-null  int64  
 18  indicator_value         178392 non-null  float64
 19  indicator_unit          178475 non-null  str    
 20  indicator_period        178475 non-null  str    
 21  comment                 178475 non-null  str    
dtypes: float64(1), int64(4), str(17)
memory usage: 157.5 MB
```

#### 3.2. Implement normalization

Варианты промптов:

(А) Простой:
```prompt
Как сделать номализацию  на Python в библиотеке Pandas? Сделай только таблицы дла записи в БД.

---
Пример данных df.head():

<ВЫВОД КОМАНДЫ df.head()>
```

(Б) Продвинутый:
```prompt
## Базовые параметры
- Роль: Junior Python Developer (Data Analyst)
- Специализация: data analysis, data visualization, data collection
- Уровень: начинающий
- Температура: 0 (максимальная точность и предсказуемость)

## Контекст выполнения
Сделать нормализацию данных на основе датафрейма.

## Входные данные
- Pandas датафрейм 

## Технические ограничения
- Использовать стандартные библиотеки Python - Pandas
- Код запускается в интерактивном ноутбуке Jupyter

## Требования к реализации

### Функциональные требования
1. Нужны отдельные таблицы на основе датафрема для каждого измерения
2. Таблицы должны быть готовы для загрузки в реляционную БД (Postgres)

### Технические требования
- Архитектура: упрощенная, для использования в интерактивных ноутбуках
- Стиль кода: PEP 8, black (длина строки 79)
- Документация: Docstrings в стиле Google
- Безопасность: Никаких захардкоженных credentials

### Структура датафрейма:

<ВЫВОД КОМАНДЫ df.info()>

### Пример данных, вывод команды df.head():

<ВЫВОД КОМАНДЫ df.head()>
```

In [ ]:
def normalize_dataframe(df):
    """
    Нормализует исходный DataFrame в набор таблиц для загрузки в БД
    """
    
    # 1. Справочник регионов
    dim_region = (df[['region_id', 'region_name']]
                  .drop_duplicates(subset=['region_id'])
                  .sort_values('region_id')
                  .reset_index(drop=True))
    
    # 2. Справочник разделов показателей
    dim_indicator_section = (df[['indicator_section_code', 'indicator_section']]
                             .drop_duplicates(subset=['indicator_section_code'])
                             .sort_values('indicator_section_code')
                             .reset_index(drop=True))
    
    # 3. Справочник показателей
    dim_indicator = (df[['indicator_code', 'indicator_name', 'indicator_section_code']]
                    .drop_duplicates(subset=['indicator_code'])
                    .sort_values('indicator_code')
                    .reset_index(drop=True))
    
    # 4. Справочник муниципальных образований (самая сложная часть)
    # Сначала создаем уникальные комбинации для муниципалитетов
    mun_columns = [
        'oktmo', 'region_id', 'mun_level', 'mun_district', 
        'municipality', 'mun_type', 'mun_type_oktmo', 'oktmo_stable',
        'oktmo_history', 'oktmo_year_from', 'oktmo_year_to'
    ]
    
    # Обрабатываем года - заменяем пустые строки на NULL
    dim_municipality = df[mun_columns].copy()
    dim_municipality['oktmo_year_from'] = pd.to_numeric(
        dim_municipality['oktmo_year_from'], 
        errors='coerce'
    ).astype('Int64')
    dim_municipality['oktmo_year_to'] = pd.to_numeric(
        dim_municipality['oktmo_year_to'], 
        errors='coerce'
    ).astype('Int64')
    
    # Заменяем NULL в year_to на 9999 (означает "до бесконечности")
    dim_municipality['oktmo_year_to'] = dim_municipality['oktmo_year_to'].fillna(9999).astype('Int64')
    
    # Удаляем дубликаты
    dim_municipality = dim_municipality.drop_duplicates().reset_index(drop=True)
    
    # Добавляем суррогатный ключ
    dim_municipality.insert(0, 'municipality_sk', range(1, len(dim_municipality) + 1))
    dim_municipality['municipality_sk'] = dim_municipality['municipality_sk'].astype('int64')
    
    # Добавляем флаг актуальности
    current_year = pd.Timestamp.now().year
    dim_municipality['is_current'] = (
        (dim_municipality['oktmo_year_from'] <= current_year) & 
        (dim_municipality['oktmo_year_to'] >= current_year)
    )
    
    # 5. Таблица фактов
    # Сначала создаем маппинг для municipality_sk
    mun_mapping = dim_municipality.set_index(['oktmo', 'oktmo_year_from', 'oktmo_year_to'])['municipality_sk'].to_dict()
    
    # Функция для поиска правильного municipality_sk
    def find_municipality_sk(row):
        oktmo = row['oktmo']
        year = row['year']
        
        # Ищем подходящую запись в dim_municipality
        mask = (
            (dim_municipality['oktmo'] == oktmo) & 
            (dim_municipality['oktmo_year_from'] <= year) & 
            (dim_municipality['oktmo_year_to'] >= year)
        )
        
        matching = dim_municipality.loc[mask, 'municipality_sk']
        return matching.iloc[0] if len(matching) > 0 else None
    
    # Создаем копию для фактов
    fact_indicator = df.copy()
    
    # Добавляем municipality_sk
    fact_indicator['municipality_sk'] = fact_indicator.apply(find_municipality_sk, axis=1)
    
    # Проверяем, что все строки получили municipality_sk
    if fact_indicator['municipality_sk'].isna().any():
        print(f"Внимание: {fact_indicator['municipality_sk'].isna().sum()} строк не получили municipality_sk")
        # Удаляем строки без маппинга
        fact_indicator = fact_indicator.dropna(subset=['municipality_sk'])
    
    # Выбираем нужные колонки для фактов
    fact_columns = [
        'municipality_sk', 'indicator_code', 'year', 
        'indicator_value', 'indicator_unit', 'indicator_period', 
        'comment', 'zagr'
    ]
    
    fact_indicator = fact_indicator[fact_columns].copy()
    
    # Добавляем суррогатный ключ для фактов
    fact_indicator.insert(0, 'fact_id', range(1, len(fact_indicator) + 1))
    
    # Обрабатываем NULL значения в indicator_value
    # (можно заменить на 0 или оставить NULL - зависит от бизнес-логики)
    fact_indicator['indicator_value'] = fact_indicator['indicator_value'].fillna(0)
    
    # Добавляем техническое поле - дата загрузки
    fact_indicator['loaded_at'] = pd.Timestamp.now()
    
    fact_indicator['municipality_sk'] = fact_indicator['municipality_sk'].astype('int64')
    
    return {
        'dim_region': dim_region,
        'dim_indicator_section': dim_indicator_section,
        'dim_indicator': dim_indicator,
        'dim_municipality': dim_municipality,
        'fact_indicator': fact_indicator
    }

In [ ]:
%%time

# Применяем нормализацию
normalized_tables = normalize_dataframe(df)

# Теперь у нас есть отдельные DataFrame для каждой таблицы
dim_region = normalized_tables['dim_region']
dim_indicator_section = normalized_tables['dim_indicator_section']
dim_indicator = normalized_tables['dim_indicator']
dim_municipality = normalized_tables['dim_municipality']
fact_indicator = normalized_tables['fact_indicator']

# Сохраняем в CSV для последующей загрузки в БД
dim_region.to_csv('dim_region.csv', index=False)
dim_indicator_section.to_csv('dim_indicator_section.csv', index=False)
dim_indicator.to_csv('dim_indicator.csv', index=False)
dim_municipality.to_csv('dim_municipality.csv', index=False)
fact_indicator.to_csv('fact_indicator.csv', index=False)

In [ ]:
# Выводим информацию о каждой таблице
print("=== Справочник регионов ===")
print(f"Размер: {dim_region.shape}")
display(dim_region.head())
print("\n")

print("=== Справочник разделов показателей ===")
print(f"Размер: {dim_indicator_section.shape}")
display(dim_indicator_section.head())
print("\n")

print("=== Справочник показателей ===")
print(f"Размер: {dim_indicator.shape}")
display(dim_indicator.head())
print("\n")

print("=== Справочник муниципальных образований ===")
print(f"Размер: {dim_municipality.shape}")
display(dim_municipality.head())
print("\n")

print("=== Таблица фактов ===")
print(f"Размер: {fact_indicator.shape}")
display(fact_indicator.head())